In [11]:
import pandas as pd
import numpy as np
import math
class Indicators():
    def __init__(self,nums=20,times=100,interval=2):
        self.df = pd.read_csv('routes4.csv',header=None)
        self.nums=nums          #节点数
        self.times=times        #时间长度
        self.interval=interval  #时间间隔
        self.count=0
        self.Accesslist=[]
        pass

    def printNodeDegree(self,time1):
        '''打印 节点度''''''打印所有节点的平均节点度'''
        arr=[]
        df=self.df
        nums=self.nums
        times=self.times
        interval=self.interval
        for num in range(0,nums):
            for time in range(interval,times,interval):
                row=df[(df[0]==str(num))&(df[1]=="+"+str(time)+"s")& (df[5]=="1")].shape[0]
                arr.append([num,time,row])
        df2=pd.DataFrame(arr)
        df2.rename(columns={0:"Node",1:"Time",2:"Degree"})
        for index,row in df2[(df2[1]==time1)].iterrows():
            print("节点"+str(row[0])+"在time:+"+str(row[1])+"s的节点度为"+str(row[2]))

    def printAvegminRoute(self):
        '''todo打印 平均最短路'''
        mylist = []
        df=self.df
        for i in range(3,101,1):
            # row=df[(df[0]==str(num))&(df[1]=="+"+str(time)+"s")& (df[5]=="1")].shape[0]
            mylist.append(df[df[1]=="+"+str(i)+"s"])
        mylist
        listmartix = []
        for j in range(len(mylist)):
            martixmap = np.zeros([20,20],dtype = int)
            for i in range(len(mylist[j])): 
                prenode = int(mylist[j].iloc[i][0])
                desnode = int(mylist[j].iloc[i][2][7:])-1
                distance = int(mylist[j].iloc[i][5])
                # print(prenode,desnode,distance)
                martixmap[prenode,desnode] = distance
                martixmap[desnode,prenode] = distance
            listmartix.append(martixmap)
        avgminroute = []
        for i in range(len(listmartix)):
            sum = np.sum(listmartix[i].reshape(-1))
            avg = sum/(20*19)
            avgminroute.append(avg.item())
        for i in avgminroute:
            print('{:.2f}'.format(i))

    def coutConnetion(self,nums,kconnect,a:list,index,time):
        '''计算time时刻k连通的个数'''
        if kconnect==0:
            flag=1
            for i in range(0,len(a)):
                for j in range(0,len(a)):
                    if (i<j and self.Accesslist[time][a[i]][a[j]]==0):
                        return
            self.count+=flag
        else:
            for i in range(nums,kconnect-1,-1):
                a[index]=i-1
                self.coutConnetion(i-1,kconnect-1,a,index+1,time)
    
    def rate(self,count,nums,k):#N取K全排列个数
        total=(math.factorial(nums)//(math.factorial(k)*math.factorial(nums-k)))
        return self.count/total
    def printKConnectivity(self,kconnect):
        df=self.df
        nums=self.nums
        name_dict={}
        times=self.times
        interval=self.interval
        dp=[]#分时间写入列表
        Accesslist=[]#各个时间段的可达矩阵列表
        for i in range(nums):
            name_dict['10.1.0.'+str(i+1)]=str(i)
        for i in range(interval,times,interval):
            dp.append(df[df[1]=="+{}s".format(str(i))])
        for i in range(len(dp)):
            AccessMatrix=[[0 for col in range(nums)] for row in range(nums)]
            for row in dp[i].itertuples():#读入邻接矩阵
                i=int(row[1])
                j=int(name_dict[row[4]])
                AccessMatrix[i][j]=1
            for k in range(nums):#warshall算法求传递闭包
                for i in range(nums):
                    for j in range(nums):
                        if AccessMatrix[i][j]==1:
                            AccessMatrix[i][j]=1
                        else:
                            AccessMatrix[i][j]=AccessMatrix[i][k]*AccessMatrix[k][j]
            Accesslist.append(AccessMatrix)
        self.Accesslist=Accesslist
        for i in range(0,49):#第2i秒
            for k in [2,kconnect,nums]:#2连通，k连通，全连通
                self.count=0
                a=[0 for col in range(k)]
                self.coutConnetion(nums,k,a,0,i)
                print('第+{}s:{}连通率:{:.4f}%'.format((i+1)*2,k,self.rate(self.count,nums,k)*100))


In [12]:
test=Indicators()
test.printAvegminRoute()

TypeError: unsupported format string passed to list.__format__